In [33]:
import pandas as pd #toolbox to work with dataframes
import numpy as np #toolbox to work with narrays
import matplotlib.pyplot as plt #toolbox to do plots
from sklearn.svm import SVC #load the support vector machine model functions
from sklearn.model_selection import train_test_split #load the function to split train and test sets
from sklearn import metrics # get the report
from sklearn.metrics import classification_report # get the report
from sklearn import preprocessing # normalize the features
from sklearn.preprocessing import MinMaxScaler # normalize the features
from sklearn.feature_selection import SelectKBest #load the feature selector model  
from sklearn.feature_selection import chi2 #feature selector algorithm


def normalized_data (df,t):

    if (t==1):
        d=df.copy() # min max normalization
        for each_collum in range(0,df.shape[1]):
            max =df.iloc[:,each_collum].max()
            min =df.iloc[:,each_collum].min()
            d.iloc[:,each_collum]=(d.iloc[:,each_collum]-min)/(max-min)
    elif (t==2):
        d=df.copy() # mean normalization
        for each_collum in range(0,df.shape[1]):
            max =df.iloc[:,each_collum].max()
            min =df.iloc[:,each_collum].min()
            mean =df.iloc[:,each_collum].mean()
            d.iloc[:,each_collum]=(d.iloc[:,each_collum]-mean)/(max-min)
    
    else:
        d=df.copy() # standardization
        for each_collum in range(0,df.shape[1]):
            mean =df.iloc[:,each_collum].mean()
            std =df.iloc[:,each_collum].std()
            d.iloc[:,each_collum]=(d.iloc[:,each_collum]-mean)/(std)

    return d

    

In [34]:
# 1st step database opening
df = pd.read_csv('winequality-white.csv',';') #open the database

# split the database in target and features
target=df.iloc[:,-1]
df=df.iloc[:,:-1]

C:\Users\prodrigues\AppData\Local\Temp\ipykernel_17632\31263044.py:2: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df = pd.read_csv('winequality-white.csv',';') #open the database


In [35]:
# 2nd step - features normalization
d_n=normalized_data (df,1)

In [36]:
# 3rd step - load and design the classifiers

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import BaggingClassifier,ExtraTreesClassifier,RandomForestClassifier,AdaBoostClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.gaussian_process.kernels import RBF
classifiers = [
    SVC(gamma='auto'),
    GaussianProcessClassifier(1.0* RBF(1.0)),
    LinearSVC(),
    SGDClassifier(),
    KNeighborsClassifier(),
    LogisticRegression(solver='lbfgs'),
    LogisticRegressionCV(cv=3),
    BaggingClassifier(),
    ExtraTreesClassifier(n_estimators=300),
    RandomForestClassifier(max_depth=5, n_estimators=300, max_features=1),
    GaussianNB(),
    DecisionTreeClassifier(max_depth=5),
    MLPClassifier(alpha=1,max_iter=1000),
    AdaBoostClassifier(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis(),
    OneVsRestClassifier(LinearSVC(random_state=0)),
    GradientBoostingClassifier(),
    SGDClassifier(),
]






In [37]:
# split data for training and testing your model
X_train, X_test, y_train, y_test = train_test_split(d_n, target, train_size = 0.75) # 75% of data goes for training and 25% goes for testing

print(X_train)


      fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
346        0.173077          0.254902     0.060241        0.010736   0.065282   
1862       0.336538          0.049020     0.186747        0.026074   0.133531   
3529       0.317308          0.264706     0.162651        0.038344   0.074184   
397        0.240385          0.117647     0.240964        0.013804   0.083086   
1986       0.365385          0.078431     0.246988        0.019939   0.112760   
...             ...               ...          ...             ...        ...   
3161       0.221154          0.088235     0.253012        0.222393   0.071217   
1141       0.528846          0.254902     0.295181        0.102761   0.127596   
3075       0.250000          0.068627     0.240964        0.013804   0.097923   
4032       0.288462          0.186275     0.132530        0.263804   0.074184   
2095       0.442308          0.196078     0.240964        0.127301   0.115727   

      free sulfur dioxide  

In [38]:
# Load the function to performe feature selection
from sklearn.feature_selection import SelectKBest
#load the feature selection algorithms
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import SelectFpr
from sklearn.feature_selection import SelectFdr
from sklearn.feature_selection import SelectFwe
from sklearn.feature_selection import GenericUnivariateSelect

# function to do feature selection
def feature_selector(X_train,y_train,X_test,type,i):
    if (type == 1):
#ANOVA F-value between label/feature for classification tasks.
        bestfeatures = SelectKBest(score_func = f_classif, k=i)
    elif(type == 2):
#Mutual information for a discrete target.
        bestfeatures = SelectKBest(score_func=mutual_info_classif, k=i)
    elif(type == 3):
    #Chi-squared stats of non-negative features for classification tasks.
        bestfeatures = SelectKBest(score_func=chi2, k=i)
    elif(type == 4):
#Select features based on an estimated false discovery rate.
        bestfeatures = SelectKBest(score_func=SelectFdr, k=i)
    elif(type == 5):
#Select features based on family-wise error rate.
        bestfeatures = SelectKBest(score_func=SelectFwe, k=i)
#Perform the feature based on selected algorithm
    fit = bestfeatures.fit(X_train,y_train)
    cols_idxs = fit.get_support(indices=True)
    Xt=X_train.iloc[:,cols_idxs] # extract the best features for training
    Xteste=X_test.iloc[:,cols_idxs] # extract the best features for testing
    return Xt,Xteste

In [39]:
# Function for doing the classification
# the Function receives as inputs:
# models -> machine learning models
# Xt -> training dataframe
# Xteste -> testing dataframe
# yTrain -> training label
def classification(models,Xt,yTrain,Xteste):
    models.fit(Xt, yTrain) # function to train the model
#y_pred means label predictions of the models
    y_pred=models.predict(Xteste) # function for testing the model
    return y_pred

In [40]:
#Load libraries for checking the classification report
from sklearn import metrics
from sklearn.metrics import classification_report
# Function defined for checking the classification performance of the model
# inputs:
# y_test -> the true label
# ypred -> model's label
# outputs: classification report metrics
# accuracy, precision, recall, f1-score
def classification_reports(y_test,ypred):
    report=classification_report(y_test, ypred, output_dict=True)
    accuracy = report['accuracy']
    precision = report['macro avg']['precision']
    recall = report['macro avg']['recall']
    f1 = report['macro avg']['f1-score']
    return [accuracy, precision, recall, f1]

In [41]:
#iterative process to find the e.g. the best 3 features, 4, 5, 6,...,
# until total of features
# classification and models performance checking.

perf_results=pd.DataFrame()

for i in range(3,d_n.shape[1]):
    Xt,Xteste = feature_selector(X_train,y_train,X_test,2,i)
    ypred = classification(classifiers[3],Xt,y_train,Xteste)
    accuracy, precision, recall, f1 = classification_reports(y_test,ypred)
    perf_results[i-3]=[i,accuracy, precision, recall, f1]

perf_results=perf_results.T # how to transpose a dataframe collumns <-> rows.
perf_results.columns=['feature','accuracy','precision','recall','f1']
#print the classification of classifier 4 by using the best 3, 4, ..., total of features
print(perf_results)   

C:\Users\prodrigues\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\prodrigues\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\prodrigues\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classificatio

   feature  accuracy  precision    recall        f1
0      3.0  0.442449   0.205934  0.143611  0.089013
1      4.0  0.440816   0.062974  0.142857  0.087414
2      5.0  0.466939   0.160464  0.208048  0.180528
3      6.0  0.490612   0.142913  0.179673  0.156236
4      7.0  0.515918   0.215575  0.203059  0.194211
5      8.0  0.458776   0.333157  0.240705  0.241544
6      9.0  0.502857   0.329954  0.238127  0.249615
7     10.0  0.505306   0.255426  0.216152  0.217049


C:\Users\prodrigues\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\prodrigues\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\prodrigues\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classificatio